# Academy of Py: PyCity Schools

### Findings:

Based on the below analysis of standardized tests, we note the following:

1. Charter schools have higher average passing rates than city schools (90.4% vs. 53.7%) at comparable or lower spending per student. All top-performing schools are charter schools.

2. Higher spending per student does not automatically translate to better outcomes. In fact, in this analysis, schools with higher spending per student underperformed compared to those with lower spending per student.

3. Small and Medium schools tend to have higher passing rates, because they are all charter schools. However, the single large charter school has a much higher passing rate than comparably-sized district schools.

4. In district schools, math scores are systematically lower than the reading scores. That's something not seen in charter schools.





In [2]:
# import packages
import os
import numpy as pd
import pandas as pd

In [3]:
# get the data, make sure to use platform independent paths
schools_fname = os.path.join("data","schools_complete.csv")
students_fname = os.path.join("data","students_complete.csv")
school_data = pd.read_csv(schools_fname)
student_data = pd.read_csv(students_fname)

In [4]:
# let's take a look (always the first thing we need to do)
print(len(school_data))
school_data.head(5)

15


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [5]:
student_data.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [6]:
# let's merge the dfs
# we use a left merge, which means we'll preserve all student data, obviously
# each student will have an associated school (they wouldn't be a student otherwise!)
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# quick look
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [7]:
# but not so fast, let's check if there are any NaNs.
nans = lambda df: df[df.isnull().any(axis=1)]
nans(school_data_complete)
# looking good!

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget


In [8]:
# number of schools
n_schools = len(school_data_complete['school_name'].unique())
# number of students
n_students = school_data_complete['Student ID'].count() # why not use count once in a while?
# total budget -- use the simple school data for this
t_budget = school_data['budget'].sum()
# average math score
av_math_score = school_data_complete['math_score'].mean()
av_read_score = school_data_complete['reading_score'].mean()
# overall passing rate and math & reading passing percentages
n_students_overall_pass = len(school_data_complete.loc[ \
                        (school_data_complete['math_score'] >= 70.0) & \
                        (school_data_complete['reading_score'] >= 70.0) ])
n_students_pass_math = len(school_data_complete.loc[school_data_complete['math_score'] >= 70.0])
n_students_pass_read = len(school_data_complete.loc[school_data_complete['reading_score'] >= 70.0])
p_pass_overall = 100.0 * n_students_overall_pass / n_students
p_pass_math = 100.0 * n_students_pass_math / n_students
p_pass_read = 100.0 * n_students_pass_read / n_students

# make a nice df!!!
district_summary = pd.DataFrame()
district_summary['Total Schools'] = [n_schools]
district_summary['Total_Students'] = [f"{n_students:,}"]
district_summary['Total_Budget'] = [f"${t_budget:,}"]
district_summary['Average Math Score'] = [f"{av_math_score:.2f}"]
district_summary['Average Reading Score'] = [f"{av_read_score:.2f}"]
district_summary['% Passing Math'] = [f"{p_pass_math:.2f}"]
district_summary['% Passing Reading'] = [f"{p_pass_read:.2f}"]
district_summary['% Overall Passing Rate'] = [f"{p_pass_overall:.2f}"]
district_summary
# Note: it seems the example given is wrong in the overall passing rate!
# Also, it doesn't make sense to display many digits, so I cut all stuff to two digits.

,Total Schools,Total_Students,Total_Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,65.17


In [9]:
# now moving on to the district summary -- lots of info to collect
schools = list(school_data_complete['school_name'].unique())
school_summary = pd.DataFrame()

school_summary['School ID'] = school_data['School ID']
school_summary['School Name'] = school_data['school_name']
school_summary['School Type'] = school_data['type']
school_summary['Total Students'] = school_data['size']
school_summary['Total School Budget'] = school_data['budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = (school_data['budget']/(school_data['size'])).map('${:,.2f}'.format)

# now the tricky part starts -- getting average scors
av_math_1 = school_data_complete.groupby(['school_name']).sum()['math_score']
nnn = school_data_complete.groupby(['school_name']).count()['student_name']
av_reading = school_data_complete.groupby(['school_name']).sum()['reading_score']
xdf = pd.DataFrame(av_math_1/nnn).reset_index().rename(columns={0:'Average Math Score'})
school_summary['Average Math Score']  = xdf['Average Math Score']
xdf = pd.DataFrame(av_reading/nnn).reset_index().rename(columns={0:'Average Reading Score'})
school_summary['Average Reading Score'] = xdf['Average Reading Score']

# passing fractions
xxx = school_data_complete[ school_data_complete['math_score'] >= 70.0 ].groupby('School ID').count()['school_name']
nnn = school_data_complete.groupby(['School ID']).count()['school_name']
school_summary['% Passing Math'] = 100*xxx/nnn
xxx = school_data_complete[ school_data_complete['reading_score'] >= 70.0 ].groupby('School ID').count()['school_name']
school_summary['% Passing Reading'] = 100*xxx/nnn
xxx = school_data_complete[ (school_data_complete['reading_score'] >= 70.0) &\
                            (school_data_complete['math_score'] >= 70.0) ].groupby('School ID').count()['school_name']
school_summary['% Overall Passing Rate'] = 100*xxx/nnn
del school_summary['School ID']

# Top Performing Schools

In [13]:
school_summary.sort_values('% Overall Passing Rate',ascending=False).set_index('School Name').head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.803279,83.814988,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.682222,83.955000,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,77.289752,80.934412,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools

In [12]:
school_summary.sort_values('% Overall Passing Rate',ascending=True).set_index('School Name').head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,83.359455,83.725724,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,83.061895,83.975780,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,77.048432,81.033963,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.102592,80.746258,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,83.418349,83.848930,66.057551,81.222432,53.539172


## Math Scores by Grade

In [14]:
nine = school_data_complete.loc[ school_data_complete['grade'] == '9th'].groupby('school_name').mean()['math_score']
ten = school_data_complete.loc[ school_data_complete['grade'] == '10th'].groupby('school_name').mean()['math_score']
eleven = school_data_complete.loc[ school_data_complete['grade'] == '11th'].groupby('school_name').mean()['math_score']
twelve = school_data_complete.loc[ school_data_complete['grade'] == '12th'].groupby('school_name').mean()['math_score']
df = pd.DataFrame({'9th': nine, '10th': ten,
                   '11th': eleven,'12th': twelve})
df = df[ ['9th','10th','11th','12th']]
df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade

In [15]:
nine = school_data_complete.loc[ school_data_complete['grade'] == '9th'].groupby('school_name').mean()['reading_score']
ten = school_data_complete.loc[ school_data_complete['grade'] == '10th'].groupby('school_name').mean()['reading_score']
eleven = school_data_complete.loc[ school_data_complete['grade'] == '11th'].groupby('school_name').mean()['reading_score']
twelve = school_data_complete.loc[ school_data_complete['grade'] == '12th'].groupby('school_name').mean()['reading_score']
df = pd.DataFrame({'9th': nine, '10th': ten,
                   '11th': eleven,'12th': twelve})
df = df[ ['9th','10th','11th','12th']]
df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [16]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# make the budget per student a number again
ss_copy = school_summary.copy()
ss_copy['Per Student Budget'] = pd.to_numeric(ss_copy['Per Student Budget'].str.replace('$',''))

In [17]:
df = ss_copy.copy()
bin_indices = pd.cut(ss_copy['Per Student Budget'],spending_bins,labels=group_names)
df['Spending Ranges (Per Student)'] = bin_indices
resdf = df.groupby('Spending Ranges (Per Student)').mean()
resdf = resdf[ ['Average Math Score','Average Reading Score', '% Passing Math', 
                '% Passing Reading','% Overall Passing Rate']]
resdf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,78.752051,81.615120,93.460096,96.610877,90.369459
$585-615,80.275842,82.601359,94.230858,95.900287,90.216324
$615-645,82.226447,83.440912,75.668212,86.106569,66.112060
$645-675,79.189791,81.876383,66.164813,81.133951,53.526855


## Scores by School Size

In [18]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
df = school_summary.copy()
bin_indices = pd.cut(school_summary['Total Students'],size_bins,labels=group_names)
df['School Size'] = bin_indices
resdf = df.groupby('School Size').mean()
resdf = resdf[ ['Average Math Score','Average Reading Score', '% Passing Math', 
                '% Passing Reading','% Overall Passing Rate']]
resdf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),80.456190,82.505546,93.550225,96.099437,89.883853
Medium (1000-2000),80.878295,82.697890,93.599695,96.790680,90.621535
Large (2000-5000),80.148011,82.429660,69.963361,82.766634,58.286003


## Scores by School Type

In [19]:
df = school_summary.copy()
resdf = df.groupby('School Type').mean()
resdf = resdf[ ['Average Math Score','Average Reading Score', '% Passing Math', 
                '% Passing Reading','% Overall Passing Rate'] ]
resdf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,80.324201,82.429369,93.620830,96.586489,90.432244
District,80.556334,82.643266,66.548453,80.799062,53.672208


## Below: Extra exploration used for learning more about the data!

In [20]:
school_summary[ school_summary['School Type'] == 'Charter']

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,76.711767,81.158020,93.867121,95.854628,89.892107
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,77.289752,80.934412,93.867718,96.539641,90.582567
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.803279,83.814988,94.133477,97.039828,91.334769
8,Holden High School,Charter,427,"$248,087.00",$581.00,77.072464,80.966394,92.505855,96.252927,89.227166
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
10,Wright High School,Charter,1800,"$1,049,400.00",$583.00,76.842711,80.744686,93.333333,96.611111,90.333333
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.682222,83.955000,93.272171,97.308869,90.948012


In [21]:
school_summary[ school_summary['School Type'] == 'District']

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635.00",$655.00,77.048432,81.033963,65.683922,81.316421,53.513884
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,83.061895,83.975780,65.988471,80.739234,53.204476
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.102592,80.746258,66.752967,80.862999,53.527508
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,76.629414,81.182722,66.680064,81.933280,54.642283
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,83.359455,83.725724,66.366592,80.220055,52.988247
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,83.418349,83.848930,66.057551,81.222432,53.539172
13,Ford High School,District,2739,"$1,763,916.00",$644.00,83.274201,83.989488,68.309602,79.299014,54.289887
